Gym Wellness AI Agent: workflow

# Step 1:

1.1: load the data

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving checkins.csv to checkins.csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving profiles.csv to profiles.csv


1.2 : read the data

In [ ]:
import pandas as pd

profiles_df = pd.read_csv("profiles.csv")
checkins_df = pd.read_csv("checkins.csv")

print(profiles_df.head())
print(checkins_df.head())


   member_id     name  age sex              goal  baseline_mood  \
0          1   Rami 1   56   F       muscle gain           2.81   
1          2   Lina 2   36   F  stress reduction           3.13   
2          3  Layla 3   20   F    overall health           2.87   
3          4  Sarah 4   29   F          fat loss           3.22   
4          5  Layla 5   33   F          fat loss           3.02   

   baseline_stress  baseline_sleep activity_type  volatility  
0             3.41            7.38           low        0.87  
1             2.26            6.29          high        0.52  
2             3.43            6.63           low        0.51  
3             2.66            6.57           low        0.58  
4             2.87            5.82          high        0.90  
   member_id        date  mood  stress  fatigue  sleep_hours  \
0          1  15/04/2025     1       4        5          7.5   
1          1  16/04/2025     1       4        5          7.5   
2          1  17/04/2025   

In [ ]:
checkins_df["date"] = pd.to_datetime(checkins_df["date"])
checkins_df = checkins_df.sort_values(["member_id", "date"])


/tmp/ipython-input-2448655450.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  checkins_df["date"] = pd.to_datetime(checkins_df["date"])


1.3 feature extraction

In [ ]:
# We'll create a copy to avoid messing original if we want
feat_df = checkins_df.copy()

# Group by member
grouped = feat_df.groupby("member_id", group_keys=False)

# 7-day rolling averages
feat_df["mood_avg_7d"]    = grouped["mood"].rolling(7, min_periods=7).mean().reset_index(level=0, drop=True)
feat_df["stress_avg_7d"]  = grouped["stress"].rolling(7, min_periods=7).mean().reset_index(level=0, drop=True)
feat_df["fatigue_avg_7d"] = grouped["fatigue"].rolling(7, min_periods=7).mean().reset_index(level=0, drop=True)
feat_df["sleep_avg_7d"]   = grouped["sleep_hours"].rolling(7, min_periods=7).mean().reset_index(level=0, drop=True)

# Activity: count how many of the last 7 days had > 0 minutes
feat_df["activity_days_7d"] = (
    grouped["activity_minutes"]
    .rolling(7, min_periods=7)
    .apply(lambda x: (x > 0).sum(), raw=False)
    .reset_index(level=0, drop=True)
)

# Sleep debt relative to 7h target
feat_df["sleep_debt_7d"] = (7 - feat_df["sleep_avg_7d"]).clip(lower=0)


In [ ]:
ml_df = feat_df.dropna(subset=["mood_avg_7d", "stress_avg_7d", "fatigue_avg_7d", "sleep_avg_7d"])

print(ml_df.shape)
print(ml_df[["member_id", "date", "mood_avg_7d", "sleep_avg_7d", "activity_days_7d", "bad_day_tomorrow"]].head())


(1568, 15)
    member_id       date  mood_avg_7d  sleep_avg_7d  activity_days_7d  \
6           1 2025-04-21          1.0      7.600000               4.0   
7           1 2025-04-22          1.0      7.700000               4.0   
8           1 2025-04-23          1.0      7.857143               5.0   
9           1 2025-04-24          1.0      7.742857               4.0   
10          1 2025-04-25          1.0      7.828571               3.0   

    bad_day_tomorrow  
6                  1  
7                  1  
8                  1  
9                  1  
10                 1  


1.4: Build X and y and train your first ML model.

In [ ]:
# Choose the feature columns
feature_cols = [
    "mood_avg_7d",
    "stress_avg_7d",
    "fatigue_avg_7d",
    "sleep_avg_7d",
    "sleep_debt_7d",
    "activity_days_7d"
]
#These are your inputs for the ML model.

In [ ]:
X = ml_df[feature_cols]
y = ml_df["bad_day_tomorrow"]
X.shape, y.shape


((1568, 6), (1568,))

In [ ]:

#Step 3.3 – Train/test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


Accuracy: 0.9044585987261147

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.92      0.92       185
           1       0.89      0.88      0.88       129

    accuracy                           0.90       314
   macro avg       0.90      0.90      0.90       314
weighted avg       0.90      0.90      0.90       314

ROC-AUC: 0.9691179551644669


1.5: check for generalization

In [ ]:
y_train_pred = model.predict(X_train)
train_acc = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_acc)


Training Accuracy: 0.8947368421052632


Interpretation:
Training = 89.47%
Test = 90.4%
These two numbers are almost identical which means: The model generalizes extremely well and there is no overfitting at all.


In [ ]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(model, X, y, cv=5, scoring='roc_auc')
print("CV AUC scores:", cv_scores)
print("Mean CV AUC:", cv_scores.mean())


CV AUC scores: [0.87911167 0.9965221  0.95451498 0.99321705 0.90179474]
Mean CV AUC: 0.9450321099663871


# Step 2: Integrate ML predictions with the LLM agent


2.1: choose_strategy based on risk score

In [ ]:
def choose_strategy(risk_score: float):
    """
    Map risk_score in [0,1] to a human-readable risk_level and strategy.
    """
    if risk_score < 0.4:
        return "LOW", "performance"   # push a bit
    elif risk_score < 0.7:
        return "MEDIUM", "balanced"   # mix care + progress
    else:
        return "HIGH", "recovery"     # prioritize rest
#This is your policy, not ML.
#ML only says: “risk = 0.82”.
#You decide what to do with it.

In [ ]:
# Get a risk_score for a given row
def compute_risk_and_strategy(row, model):
    x = pd.DataFrame([row[feature_cols]])   # keeps column names
    risk_score = model.predict_proba(x)[0, 1]
    risk_level, strategy = choose_strategy(risk_score)
    return pd.Series({
        "risk_score": risk_score,
        "risk_level": risk_level,
        "strategy": strategy
    })

ml_with_risk = ml_df.copy()
ml_with_risk[["risk_score", "risk_level", "strategy"]] = (
    ml_with_risk.apply(compute_risk_and_strategy, axis=1, model=model)
)

ml_with_risk[["member_id", "date", "risk_score", "risk_level", "strategy"]].head(20)


,member_id,date,risk_score,risk_level,strategy
6,1,2025-04-21,0.963285,HIGH,recovery
7,1,2025-04-22,0.962024,HIGH,recovery
8,1,2025-04-23,0.968563,HIGH,recovery
9,1,2025-04-24,0.961471,HIGH,recovery
10,1,2025-04-25,0.949605,HIGH,recovery
11,1,2025-04-26,0.937635,HIGH,recovery
12,1,2025-04-27,0.938508,HIGH,recovery
13,1,2025-04-28,0.923052,HIGH,recovery
14,1,2025-04-29,0.921617,HIGH,recovery
15,1,2025-04-30,0.921978,HIGH,recovery


In [ ]:
# 1) How many LOW / MEDIUM / HIGH overall?
ml_with_risk["risk_level"].value_counts()
ml_with_risk["risk_level"].value_counts(normalize=True)  # in %

# 2) Distribution of risk scores
ml_with_risk["risk_score"].describe()


,risk_score
count,1568.000000
mean,0.411928
std,0.409062
min,0.000003
25%,0.023125
50%,0.211472
75%,0.923266
max,0.999381


In [ ]:
# Member 1 only
ml_with_risk[ml_with_risk["member_id"] == 1]["risk_level"].value_counts(normalize=True)

# Compare with a few other members
for m in [2, 3, 4, 5]:
    print("Member", m)
    print(ml_with_risk[ml_with_risk["member_id"] == m]["risk_level"].value_counts(normalize=True))
    print("------")


Member 2
risk_level
LOW       0.45
HIGH      0.40
MEDIUM    0.15
Name: proportion, dtype: float64
------
Member 3
risk_level
HIGH      0.609756
MEDIUM    0.243902
LOW       0.146341
Name: proportion, dtype: float64
------
Member 4
risk_level
MEDIUM    0.354839
HIGH      0.322581
LOW       0.322581
Name: proportion, dtype: float64
------
Member 5
risk_level
HIGH      0.693548
MEDIUM    0.290323
LOW       0.016129
Name: proportion, dtype: float64
------


2.2: Build a textual summary from features (no LLM yet)

In [ ]:
def summarize_features_for_llm(row, profile_row):
    """
    Turn numeric 7-day features into a short textual summary.
    This is still pure Python, no LLM yet.
    """
    mood = row["mood_avg_7d"]
    stress = row["stress_avg_7d"]
    fatigue = row["fatigue_avg_7d"]
    sleep = row["sleep_avg_7d"]
    sleep_debt = row["sleep_debt_7d"]
    activity_days = row["activity_days_7d"]

    name = profile_row["name"]
    age = profile_row["age"]
    goal = profile_row["goal"]

    mood_desc = "generally good" if mood >= 3.5 else "moderate" if mood >= 2.5 else "rather low"
    stress_desc = "low" if stress <= 2.5 else "moderate" if stress <= 3.5 else "high"
    fatigue_desc = "low" if fatigue <= 2.5 else "moderate" if fatigue <= 3.5 else "high"
    sleep_desc = "quite short" if sleep <= 6 else "adequate" if sleep <= 7.5 else "quite long"
    activity_desc = "very little" if activity_days <= 1 else "some" if activity_days <= 4 else "regular"

    summary = (
        f"Member: {name}, {age} years old. Goal: {goal}.\n"
        f"Over the last 7 days, mood was {mood_desc}, stress was {stress_desc}, "
        f"and fatigue was {fatigue_desc}.\n"
        f"Sleep has been {sleep_desc} with about {sleep:.1f} hours on average "
        f"and a sleep debt of roughly {sleep_debt:.1f} hours.\n"
        f"There was {activity_desc} physical activity (active on about {activity_days:.0f} days)."
    )

    return summary


In [ ]:
#Test it on one example row (no LLM yet):
example_row = ml_with_risk.iloc[0]
member_id = example_row["member_id"]

profile_row = profiles_df[profiles_df["member_id"] == member_id].iloc[0]

print(summarize_features_for_llm(example_row, profile_row))
print("Risk level:", example_row["risk_level"], "Risk score:", round(example_row["risk_score"], 2))


Member: Rami 1, 56 years old. Goal: muscle gain.
Over the last 7 days, mood was rather low, stress was high, and fatigue was high.
Sleep has been quite long with about 7.6 hours on average and a sleep debt of roughly 0.0 hours.
There was some physical activity (active on about 4 days).
Risk level: HIGH Risk score: 0.96


 → this will be the input context for the LLM.

In [ ]:
!pip install --quiet openai


# Step 3: Stub LLM functions (NO real OpenAI yet)

These functions pretend to be the LLM for now.
Later we will replace the inside with real API calls.

 LLM stubs:

llm_explain_risk_stub

llm_generate_plan_stub

WellnessAgent class:

run_for_member(member_id, today_date)

run_for_all_members(today_date)

 daily_plans_df built and column order set

In [ ]:
#3.1) Risk explanation stub
def llm_explain_risk_stub(row, profile_row):
    """
    TEMPORARY stub.
    Later this will call OpenAI.
    For now, we return a fake explanation string.
    """
    risk_level = row["risk_level"]
    summary_text = summarize_features_for_llm(row, profile_row)

    # For debugging / report now, we just build a simple text.
    explanation = (
        f"(STUB) Explanation for risk level {risk_level}.\n"
        f"Based on the recent week, it seems that mood, stress, sleep, and activity "
        f"patterns may be making tomorrow more challenging.\n"
        f"This will later be generated by the LLM using this summary:\n\n"
        f"{summary_text}"
    )
    return explanation


In [ ]:
#3.2) Micro-plan stub
def llm_generate_plan_stub(row, profile_row, risk_explanation):
    """
    TEMPORARY stub.
    Later this will be replaced with an OpenAI call that creates a 3-day micro-plan.
    For now, we just return a simple hard-coded plan.
    """
    name = profile_row["name"]
    goal = profile_row["goal"]
    risk_level = row["risk_level"]
    strategy = row["strategy"]

    plan = (
        f"(STUB PLAN) Hi {name}, over the next 3 days we will follow a {strategy} strategy "
        f"to support your goal of {goal}.\n\n"
        "Day 1: Aim for an earlier bedtime, a short light walk, and 10 minutes of deep breathing.\n"
        "Day 2: Keep movement gentle (walk or easy mobility) and schedule one relaxing activity you enjoy.\n"
        "Day 3: Maintain regular sleep and hydration, and write down one small thing that went well each day."
    )

    return plan
#Later you’ll just keep the same function names but replace the inside with actual client.responses.create(...).

In [ ]:
#3.3 wellnessagent all together
class WellnessAgent:
    def __init__(self, ml_df_with_risk, profiles_df, model, feature_cols):
        self.ml_df = ml_df_with_risk
        self.profiles_df = profiles_df
        self.model = model
        self.feature_cols = feature_cols

    def _get_row_for_member_and_date(self, member_id, today_date):
        # today_date is a pandas.Timestamp or string "YYYY-MM-DD"
        df = self.ml_df.copy()
        df["date"] = pd.to_datetime(df["date"])
        today_date = pd.to_datetime(today_date)

        row = df[(df["member_id"] == member_id) & (df["date"] == today_date)]
        if row.empty:
            return None
        return row.iloc[0]

    def run_for_member(self, member_id, today_date):
        row = self._get_row_for_member_and_date(member_id, today_date)
        if row is None:
            return None  # no data / no 7 days history

        profile_row = self.profiles_df[self.profiles_df["member_id"] == member_id].iloc[0]

        # In the future we could recompute risk here; for now we use the stored risk_score/level/strategy
        risk_score = row["risk_score"]
        risk_level = row["risk_level"]
        strategy = row["strategy"]

        # === LLM STUBS (replace later with real calls) ===
        risk_explanation = llm_explain_risk_stub(row, profile_row)
        plan_text = llm_generate_plan_stub(row, profile_row, risk_explanation)

        return {
            "member_id": member_id,
            "name": profile_row["name"],
            "date_for_plan": pd.to_datetime(today_date).strftime("%Y-%m-%d"),
            "risk_score": float(risk_score),
            "risk_level": risk_level,
            "strategy": strategy,
            "risk_explanation": risk_explanation,
            "plan_text": plan_text,
        }

    def run_for_all_members(self, today_date):
        results = []
        member_ids = self.ml_df["member_id"].unique()
        for m in member_ids:
            out = self.run_for_member(m, today_date)
            if out is not None:
                results.append(out)
        return pd.DataFrame(results)


In [ ]:
#Create an instance:
feature_cols = [
    "mood_avg_7d",
    "stress_avg_7d",
    "fatigue_avg_7d",
    "sleep_avg_7d",
    "sleep_debt_7d",
    "activity_days_7d",
]

agent = WellnessAgent(
    ml_df_with_risk=ml_with_risk,
    profiles_df=profiles_df,
    model=model,
    feature_cols=feature_cols,
)


In [ ]:
#Test for one member and one date (still stubbed, no LLM):
test_member = 1
test_date = "2025-05-05"   # make sure this date exists in ml_with_risk for member 1

result = agent.run_for_member(test_member, test_date)
result


{'member_id': 1,
 'name': 'Rami 1',
 'date_for_plan': '2025-05-05',
 'risk_score': 0.9358533921119832,
 'risk_level': 'HIGH',
 'strategy': 'recovery',
 'risk_explanation': '(STUB) Explanation for risk level HIGH.\nBased on the recent week, it seems that mood, stress, sleep, and activity patterns may be making tomorrow more challenging.\nThis will later be generated by the LLM using this summary:\n\nMember: Rami 1, 56 years old. Goal: muscle gain.\nOver the last 7 days, mood was rather low, stress was high, and fatigue was high.\nSleep has been quite long with about 7.8 hours on average and a sleep debt of roughly 0.0 hours.\nThere was some physical activity (active on about 2 days).',
 'plan_text': '(STUB PLAN) Hi Rami 1, over the next 3 days we will follow a recovery strategy to support your goal of muscle gain.\n\nDay 1: Aim for an earlier bedtime, a short light walk, and 10 minutes of deep breathing.\nDay 2: Keep movement gentle (walk or easy mobility) and schedule one relaxing acti

In [ ]:
daily_plans_df = agent.run_for_all_members("2025-05-05")
daily_plans_df.head()
#This DataFrame is exactly what you will later export to:CSV, Google Sheets ,Make.com scenario

,member_id,name,date_for_plan,risk_score,risk_level,strategy,risk_explanation,plan_text
0,1,Rami 1,2025-05-05,0.935853,HIGH,recovery,(STUB) Explanation for risk level HIGH.\nBased...,"(STUB PLAN) Hi Rami 1, over the next 3 days we..."
1,2,Lina 2,2025-05-05,0.146638,LOW,performance,(STUB) Explanation for risk level LOW.\nBased ...,"(STUB PLAN) Hi Lina 2, over the next 3 days we..."
2,3,Layla 3,2025-05-05,0.781774,HIGH,recovery,(STUB) Explanation for risk level HIGH.\nBased...,"(STUB PLAN) Hi Layla 3, over the next 3 days w..."
3,4,Sarah 4,2025-05-05,0.437857,MEDIUM,balanced,(STUB) Explanation for risk level MEDIUM.\nBas...,"(STUB PLAN) Hi Sarah 4, over the next 3 days w..."
4,5,Layla 5,2025-05-05,0.683472,MEDIUM,balanced,(STUB) Explanation for risk level MEDIUM.\nBas...,"(STUB PLAN) Hi Layla 5, over the next 3 days w..."


In [ ]:
# Ensure column order is standard
daily_plans_df = daily_plans_df[
    [
        "date_for_plan", "member_id", "name",
        "risk_score", "risk_level", "strategy",
        "risk_explanation", "plan_text"
    ]
]


Nice small “wrap-up” helper:
To make your life easy later when you plug in the LLM, you can create one helper that runs everything for a given date and saves the CSV:


In [ ]:
def run_pipeline_for_date(agent, date_str):
    """
    Runs the wellness agent for all members on a given date
    and saves the daily plans as a CSV file.
    Currently uses stub LLM outputs.
    """
    daily_plans_df = agent.run_for_all_members(date_str)

    if daily_plans_df.empty:
        print(f"No members with enough history on {date_str}.")
        return None

    # Ensure column order
    daily_plans_df = daily_plans_df[
        [
            "date_for_plan", "member_id", "name",
            "risk_score", "risk_level", "strategy",
            "risk_explanation", "plan_text"
        ]
    ]

    filename = f"daily_plans_{date_str}.csv"
    daily_plans_df.to_csv(filename, index=False)
    print("Saved:", filename)

    return daily_plans_df


# step 4: Implimentation of llm


In [ ]:

import os

# 🔴  run ONCE per session
os.environ["OPENAI_API_KEY"] = "your API key here "


In [ ]:
#Cell 3 – Create client
from openai import OpenAI

client = OpenAI()  # It will automatically read OPENAI_API_KEY


In [ ]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="Say hello as a gym wellness coach in one short friendly sentence."
)

print(response.output[0].content[0].text)


Hey there! Ready to crush your wellness goals together? 💪😊


 4.2: Real llm_explain_risk (replaces stub)

In [ ]:
def llm_explain_risk_v2(row, profile_row):
    summary_text = summarize_features_for_llm(row, profile_row)
    risk_level = row["risk_level"]

    prompt = f"""
You are an experienced gym wellness coach writing an INTERNAL note
to another coach. The member will NOT read this text.

7-day member summary:
{summary_text}

Internal assessment: The member is at a {risk_level} level for tomorrow.

Write:
- 2–3 short bullet points explaining WHY this member may be struggling
- Reference mood, stress, sleep, fatigue, and activity patterns
- No numbers, probabilities, or the word “risk”
- Professional, supportive tone
- This is for the COACH only
"""

    response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt
    )

    return response.output[0].content[0].text.strip()


4.3: Real llm_generate_plan (3-day micro-plan)

In [ ]:
# Fixed list of gym services we allow the LLM to choose from
FIXED_OFFERS = [
    "yoga class",
    "mobility class",
    "pilates session",
    "personal training session",
    "nutrition consultation",
    "stress-coaching session"
]


def llm_generate_plan_v2(row, profile_row, risk_explanation):
    summary_text = summarize_features_for_llm(row, profile_row)

    name = profile_row["name"]
    goal = profile_row["goal"]
    risk_level = row["risk_level"]
    strategy = row["strategy"]

    # Build the prompt with the fixed offers list inserted
    prompt = f"""
You are a compassionate gym wellness coach.

Member: {name}
Goal: {goal}

7-day summary:
{summary_text}

Internal assessment: {risk_level} – strategy "{strategy}"

Coach explanation:
{risk_explanation}

Write the output in EXACTLY this structure:

PLAN:
(1–2 gentle sentences speaking directly to {name})
(Day 1: sleep target + activity suggestion + relaxation technique)
(Day 2: sleep target + activity suggestion + relaxation technique)
(Day 3: sleep target + activity suggestion + relaxation technique)

OFFERS:
Choose 1–3 items ONLY from the following FIXED LIST of gym services:
{", ".join(FIXED_OFFERS)}

Important rules:
- You MUST choose ONLY items from the fixed list above.
- Do NOT invent any new services.
- Write ONLY a comma-separated list after "OFFERS:".
- NO extra sentences, NO explanations, NO full stops.
"""

    response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt
    )

    full_text = response.output[0].content[0].text.strip()

    # Extract plan and offers
    if "OFFERS:" in full_text:
        plan_part, offers_part = full_text.split("OFFERS:", 1)
        plan_text = plan_part.replace("PLAN:", "").strip()
        offer_suggestions = offers_part.strip()  # clean comma list for Power BI
    else:
        plan_text = full_text
        offer_suggestions = ""

    return plan_text, offer_suggestions


4.4: Redefine WellnessAgent to use real LLM


In [ ]:
class WellnessAgentV2:
    def __init__(self, ml_df_with_risk, profiles_df, model, feature_cols):
        self.ml_df = ml_df_with_risk
        self.profiles_df = profiles_df
        self.model = model
        self.feature_cols = feature_cols

    def _get_row(self, member_id, today_date):
        df = self.ml_df.copy()
        df["date"] = pd.to_datetime(df["date"])
        today_date = pd.to_datetime(today_date)
        row = df[(df["member_id"] == member_id) & (df["date"] == today_date)]
        return None if row.empty else row.iloc[0]

    def run_for_member(self, member_id, today_date):
        row = self._get_row(member_id, today_date)
        if row is None:
            return None

        profile_row = self.profiles_df[self.profiles_df["member_id"] == member_id].iloc[0]

        risk_score = row["risk_score"]
        risk_level = row["risk_level"]
        strategy = row["strategy"]

        # LLM v2
        risk_explanation = llm_explain_risk_v2(row, profile_row)
        plan_text, offer_suggestions = llm_generate_plan_v2(row, profile_row, risk_explanation)

        return {
            "member_id": member_id,
            "name": profile_row["name"],
            "date_for_plan": today_date,
            "risk_score": float(risk_score),
            "risk_level": risk_level,
            "strategy": strategy,
            "risk_explanation": risk_explanation,
            "plan_text": plan_text,
            "offer_suggestions": offer_suggestions,
        }

    def run_for_all_members(self, today_date):
        results = []
        for member_id in self.ml_df["member_id"].unique():
            out = self.run_for_member(member_id, today_date)
            if out is not None:
                results.append(out)
        return pd.DataFrame(results)


In [ ]:
feature_cols = [
    "mood_avg_7d", "stress_avg_7d", "fatigue_avg_7d",
    "sleep_avg_7d", "sleep_debt_7d", "activity_days_7d"
]

agent_v2 = WellnessAgentV2(ml_with_risk, profiles_df, model, feature_cols)



In [ ]:
daily_plans_v2 = agent_v2.run_for_all_members("2025-05-05")
daily_plans_v2.head()



,member_id,name,date_for_plan,risk_score,risk_level,strategy,risk_explanation,plan_text,offer_suggestions
0,1,Rami 1,2025-05-05,0.935853,HIGH,recovery,- Rami's low mood combined with high stress le...,"**** \nRami, let's focus on gentle movements ...","** yoga class, personal training session, stre..."
1,2,Lina 2,2025-05-05,0.146638,LOW,performance,### Internal Note for Coach\n\n- Lina's consis...,"Lina, let’s focus on enhancing your rest and r...","yoga class, stress-coaching session, nutrition..."
2,3,Layla 3,2025-05-05,0.781774,HIGH,recovery,**Internal Notes for Coach: Layla 3**\n\n- Lay...,"Hey Layla, let's focus on gentle strategies to...","yoga class, mobility class, stress-coaching se..."
3,4,Sarah 4,2025-05-05,0.437857,MEDIUM,balanced,- Sarah's low mood and high stress may be cont...,"****\nSarah, I understand things have been tou...","** yoga class, stress-coaching session, nutrit..."
4,5,Layla 5,2025-05-05,0.683472,MEDIUM,balanced,- Layla’s low mood combined with high stress l...,"Layla, it’s important to be gentle with yourse...","yoga class, mobility class, nutrition consulta..."


In [ ]:
daily_plans_v2 = daily_plans_v2[
    [
        "date_for_plan", "member_id", "name",
        "risk_score", "risk_level", "strategy",
        "risk_explanation", "plan_text",
        "offer_suggestions",
    ]
]


In [ ]:
daily_plans_v2.to_csv("daily2_plans2_2025-05-05.csv", index=False)

In [ ]:
from google.colab import files
files.download("daily2_plans2_2025-05-05.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>